## 1. 빅쿼리에서 데이터 불러오기

In [3]:
import pandas as pd
import json
from pandas.io import gbq
from pandas import DataFrame

In [20]:
from typing import List
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [4]:
from google.colab import auth
auth.authenticate_user()
project_id = "dbmsproject-350608"  # replace with GCP project to bill against
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [5]:
sql = "SELECT * FROM part1json.prac1"
project_id = 'dbmsproject-350608' # Project id
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

## 2. Text embedding 만들기

In [ ]:
df2 = df.reset_index()
all_titles = [df2['title'][i] for i in range(len(df2['title']))]

In [63]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4", tags=None, options=None)

In [13]:
title_lst = list(df['title'])

In [64]:
embeddings = embed(title_lst)

In [65]:
print(embeddings.shape)

(72, 512)


In [66]:
embeddings

<tf.Tensor: shape=(72, 512), dtype=float32, numpy=
array([[-0.0128849 ,  0.03435858,  0.04579103, ..., -0.01481417,
         0.04610016,  0.03768883],
       [-0.0128849 ,  0.03435858,  0.04579103, ..., -0.01481417,
         0.04610016,  0.03768883],
       [-0.0128849 ,  0.03435858,  0.04579103, ..., -0.01481417,
         0.04610016,  0.03768883],
       ...,
       [ 0.02569636, -0.03797903,  0.00284038, ..., -0.06915897,
         0.0363395 , -0.08800146],
       [-0.0128849 ,  0.03435858,  0.04579103, ..., -0.01481417,
         0.04610016,  0.03768883],
       [-0.01749272, -0.00035105, -0.06476826, ..., -0.05020081,
        -0.01653336, -0.08876848]], dtype=float32)>

In [74]:
emb_lst = embeddings.tolist()

In [ ]:
emb_lst[0]

In [57]:
emb_lst = list(embeddings)

### Bigquery SQL 사용하게 될 경우 사용할 코드. 사용하지 않을 경우 아래의 파이썬 추천 코드 사용

In [ ]:
#standardSQL
'''
SELECT
  c.k1 as input_article_title,
  c.k2 as similar_article_title,
  SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity
FROM
(
  SELECT
    a.key k1, a.val v1, b.key k2, b.val v2
  FROM
  (
    SELECT title key, title_embed val
    FROM reuters.embeddings
    WHERE title LIKE "HIGH WINDS KEEP VESSELS TRAPPED IN BALTIC ICE"
    LIMIT 1

   ) a
  CROSS JOIN
  (
    SELECT title key, title_embed val
    FROM reuters.embeddings
  ) b
) c
, UNNEST(c.v1) vv1 with offset ind1 JOIN UNNEST(c.v2) vv2 with offset ind2 ON (ind1=ind2)
GROUP BY c.k1, c.k2
ORDER BY similarity DESC
LIMIT 10
'''

In [ ]:
'''
SELECT
  title,
  content,
  title_embed,
  content_embed
FROM
  reuters.embeddings
LIMIT 1
'''

## 3. 추천 모델 생성

In [43]:
def get_recommendations(title):
  title_num = title_lst.index(title)
  tit_lst, score_lst = [], []
  for i in range(len(title_lst)):
    tit_lst.append(title_lst[i])
    score_lst.append(np.inner(embeddings[title_num], embeddings[i]))
  del tit_lst[title_num]
  del score_lst[title_num]
  dft = pd.DataFrame(tit_lst, columns = ['Title'])
  dfs = pd.DataFrame(score_lst, columns = ['Score'])
  return_df = pd.concat([dft,dfs],axis=1)
  return_df = return_df.sort_values('Score', ascending=False)
  return_df = return_df[0:10]
  return return_df

In [45]:
get_recommendations('3GIO.')

,Title,Score
22,Frontmatter,0.553950
5,Flußanalyse,0.550750
62,FFTW.,0.544288
34,Fachkunde,0.542142
20,1may,0.536059
41,FGDC.,0.535840
10,7373170279850,0.529005
16,Flexonics.,0.518328
23,Fractalmania,0.486325
47,2BTextures,0.479337


In [ ]:
import flask
app = flask.Flask(__name__, template_folder='templates')
# Set up the main route
@app.route('/', methods=['GET', 'POST'])
def main():
  if flask.request.method == 'GET':
    return(flask.render_template('index.html'))
  if flask.request.method == 'POST':
    pname = flask.request.form['paper_name']
    pname = pname.title()
    if pname not in all_titles:
      return(flask.render_template('negative.html', name=pname))
    else:
      result_final = get_recommendations(pname)
      names, dates = [], []
      for i in range(len(result_final)):
        names.append(result_final.iloc[i][0])
        dates.append(result_final.iloc[i][1])
      return flask.render_template('positive.html', paper_names=names, paper_date=dates, search_name=pname)
    
if __name__ == '__main__':
    app.run()